In [1]:
#Spliting train and test correctly (GPT)
import os
import numpy as np
import matplotlib.pyplot as plt
import mne
import pyxdf
import pandas as pd
import re
import mne
from sklearn.decomposition import fastica
from pyprep import PrepPipeline, NoisyChannels
from matplotlib.colors import TwoSlopeNorm
import matplotlib
from mne_icalabel import label_components
from autoreject import AutoReject
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
from sklearn.metrics import make_scorer, fbeta_score
from sklearn.svm import SVC
%matplotlib qt
matplotlib.use('Qt5Agg')
mne.set_log_level('warning')

In [2]:
# some function to make the end result more readable:
#filename has to be the entire path to the file so "data//P101//filename"

def get_data(filename, pilot=False):    # Pilot = True for all pilots except Pilot117
    print('Reading data')

    # Because of the software switch there are now 3 ACC channels and one Marker channel
    nr_non_eeg = 4
    if pilot:
        nr_non_eeg = 3

    # Reading in the xdf files and extracting the marker and eeg streams
    streams, fileheader = pyxdf.load_xdf(filename, select_streams=[{'type': 'EEG'}, {'name':'LSL4Unity.OmnideckWaiterVR'}] , synchronize_clocks=False)
    marker_stream = next(s for s in streams if 'LSL4Unity.OmnideckWaiterVR' in s['info']['name'][0])
    eeg_stream = next(s for s in streams if "EEG" in s['info']['type'][0])
    eeg_data = np.array(eeg_stream['time_series']).T
    #eeg_timestamps = np.array(eeg_stream['time_stamps'])
    sfreq = float(eeg_stream['info']['nominal_srate'][0])

    # Collection all ch names and renaming TP9, TP10
    ch_names = []
    for ch in eeg_stream['info']['desc'][0]['channels'][0]['channel']:
        if ch['label'][0] == 'TP10':
            ch_names.append('FCz')
        elif ch['label'][0] == 'TP9':
            ch_names.append('Fpz')
        elif ch['label'][0] == 'FPz':
            ch_names.append('Fpz')
        else:
            ch_names.append(ch['label'][0])
    info = mne.create_info(
        ch_names=ch_names[:-nr_non_eeg],
        sfreq=sfreq,
        ch_types='eeg'
    )
    raw = mne.io.RawArray(eeg_data[:-nr_non_eeg]/10e5, info)

    # Creating the Montage
    montage = mne.channels.make_standard_montage('standard_1020')
    raw.set_montage(montage)

    # Calculating the event samples for the annotations
    event_samples = (marker_stream['time_stamps'] - eeg_stream['time_stamps'][0])*sfreq
    event_samples = event_samples.astype(int)
    event_labels = [int(marker[0]) for marker in marker_stream['time_series']]


    annotations = mne.Annotations(onset=event_samples / sfreq,
                                  duration=[0] * len(event_samples),  # Instantaneous events
                                  description=list(event_labels))
    raw.set_annotations(annotations)

    return raw, marker_stream, event_samples

#apply a notch filter at 50hz and filter between 0.01 and 40Hz. Filter above 1Hz is applied when creating the epochs
def filter_data(raw):
    # Notch Filter at 50 Hz
    raw = raw.notch_filter(50, method='fir', phase='zero',verbose=False)
    # Filter data between 0.01 and 40 Hz
    iir_params = dict(order=2, ftype='butter',verbose=False)
    raw = raw.filter(l_freq=0.01, h_freq=40, method='iir', iir_params=iir_params, phase='zero', verbose=False)
    return raw


# making epochs around a specified marker, Applying 1Hz Highpass filter if epoch is used for ICA
def get_epochs(raw: mne.io.Raw, marker_stream, event_samples , marker_id:int, tmin:int, tmax:int, preload=False, ica=True):
    current_sfreq = raw.info["sfreq"]
    desired_sfreq = 128  # Hz
    decim = np.round(current_sfreq / desired_sfreq).astype(int)
    if ica:
        iir_params = dict(order=2, ftype='butter',verbose=False)
        raw = raw.copy()
        raw = raw.filter(l_freq=1, h_freq=None, method='iir', iir_params=iir_params, phase='zero', verbose=False)
    event_labels = [int(marker[0]) for marker in marker_stream['time_series']]
    events = np.array([[sample, 0, label] for sample, label in zip(event_samples, event_labels)])
    selected_events = events[events[:, 2] == marker_id]
    epochs = mne.Epochs(raw, np.array(selected_events), event_id=int(marker_id),baseline=None, tmin=tmin, tmax=tmax, reject_by_annotation=False, verbose=False, preload=preload, decim=decim)
    return epochs

# Counts the markers and saves them in a file.
def count_markers(marker_stream, match = False):
    flat_list = [int(marker[0]) for marker in marker_stream['time_series']]
    marker_count = {}
    for marker in flat_list:
        if marker in marker_count:
            marker_count[marker] += 1
        else:
            marker_count[marker] = 1
    marker_count = dict(sorted(marker_count.items()))
    if match:
        with open(f'markers\\{match}_markers.txt', 'w') as file:
            for key, value in marker_count.items():
                file.write(f"{key}: {value}\n")
    #return marker_count

# ICA applied to individual epochs. Outdated function
def apply_ica_indiv(epochs: mne.Epochs):
    cleaned_epochs = []
    all_rej = []
    i = 0
    while i < np.shape(epochs.events)[0]:
        epoch = epochs[i]
        ica = mne.preprocessing.ICA(n_components=32, method='fastica', random_state=42)
        ica.fit(epoch)
        source = ica.get_sources(epoch).get_data()
        std = np.std(source)
        rej_ch = []
        for j, channel in enumerate(source[0]):
            if np.max(channel) > 5*std:     # This has been chosen somewhat arbitrarily now as it results in
                rej_ch.append(j)
        all_rej.append(rej_ch)

        cleaned_epochs.append(ica.apply(epoch, exclude=rej_ch))
        i+=1
    cleaned_epochs = mne.EpochsArray(
        data = np.squeeze([e.get_data() for e in cleaned_epochs]),
        info = epochs.info,
        events= epochs.events
    )
    return cleaned_epochs

# Naive way of ICA component rejection, based on maximum amplitude of component.
def get_ica(epochs: mne.Epochs, n_components=32, plot=False, save_path=None, match=None):
    ica = mne.preprocessing.ICA(n_components=n_components, max_iter= 1500, method='fastica', verbose=False)
    ica.fit(epochs, verbose=False)
    #std = np.std(ica.get_sources(epochs).get_data())
    rej_ch = []
    for i, channel in enumerate(ica.get_sources(epochs).get_data()[0]):
        std = np.std(channel)
        if np.max(np.absolute(channel)) > 6*std:
            rej_ch.append(i)
    if plot and len(rej_ch) != 0:
        #print(len(rej_ch))
        with open(f'figures\\meeting\\ica\\{match}.txt', 'w') as file:
            file.write(", ".join(map(str, rej_ch)))
            file.write(f"\n{len(rej_ch)} rejected channels")
            file.close()
        fig = ica.plot_components(title=f'{match} all ICA components', show=False);
        for j, f in enumerate(fig):
            f.savefig(save_path+f'{match}_{j}_all_ica.png')
            plt.close(f)
        fig = ica.plot_components(picks=rej_ch,title=f'{match} rejected channels', show=False);
        fig.savefig(save_path+f'{match}_rejected_ica.png')
        plt.close(fig)
    return ica, rej_ch
    #ica.apply(epochs, exclude=rej_ch, verbose=False)

# ICA method using the mne implementation of icalabel
def get_icalabel(epochs: mne.Epochs, n_components=32, plot=False, save_path=None, match=None):
    """"
    A function extracting the ICA components and rejecting all components that are not labeled Brain by the icalabel function. Using a certainty threshhold of 70%

    epochs | mne epoch for which to calculate the ICA \n
    n_components | number of ICA components \n
    plot | whether to plot the ICA components \n
    save_path | path to save the ICA components \n
    match | the match regex that indicates the participant, trial and condition
    """
    ica = mne.preprocessing.ICA(n_components=n_components, max_iter= 500, method='infomax', fit_params=dict(extended=True), verbose=False)
    ica.fit(epochs, verbose=False)
    rej_ch = []
    a = label_components(epochs, ica, method='iclabel')
    for i, ic in enumerate(a['labels']):
        #print(f'Component {i} is {ic} with proba {a['y_pred_proba'][i]}')
        if ic != 'brain':       #and a['y_pred_proba'][i] > 0.7
            rej_ch.append(i)

    # Ploitting the Components as well as Sources
    print('Rejected components:', len(rej_ch))
    if plot and len(rej_ch) != 0:
        # Saving Rejected channelsin txt
        with open(f'figures\\meeting\\ica\\{match}.txt', 'w') as file:
            file.write(", ".join(map(str, rej_ch)))
            file.write(f"\n{len(rej_ch)} rejected channels")
            file.close()

        # Plotting Components
        fig = ica.plot_components(title=f'{match} all ICA components', show=False);
        for j, f in enumerate(fig):
            f.savefig(save_path+f'{match}_{j}_all_ica.png')
            plt.close(f)
        fig = ica.plot_components(picks=rej_ch,title=f'{match} rejected channels', show=False);
        fig.savefig(save_path+f'{match}_rejected_ica.png')
        plt.close(fig)

        # Plotting Sources
        fig = ica.plot_sources(epochs, picks=slice(0,32,1))
        fig.savefig(save_path+f'{match}_ica_sources.png')
        plt.close(fig)
    return ica, rej_ch

# load data, filter, epoch, ICA, average, plot
def mrcp(epochs):
    frontal_channels = ['F3', 'Fz', 'F4', 'FC1', 'FCz', 'FC2', 'C3', 'Cz', 'C4', 'CP1', 'CP2']
    iir_params = dict(order=2, ftype='butter')
    epochs = epochs.filter(l_freq =0.1 ,h_freq=1, method='iir', iir_params=iir_params, phase='zero')
    evoked = epochs.average()
    evoked = evoked.pick(frontal_channels)
    return evoked


def drop_bad_epochs(epochs):
    epoch_data = epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)
    std_per_channel = np.std(epoch_data, axis=(0, 2))  # Standard deviation per channel
    threshold = 10 * std_per_channel[:, np.newaxis]  # Expand dims to match shape
    # Find epochs where any channel exceeds the threshold
    bad_epochs = np.any(np.abs(epoch_data) > threshold[np.newaxis, :, :], axis=(1, 2))
    print(f'Bad epochs: {len(bad_epochs)}')
    # Drop bad epochs
    epochs_clean = epochs[~bad_epochs]
    return epochs_clean

# Given an epochs object, extracts the data for each 1s long window with a stepsize of 125ms. Creating 57 windwos (-4, 4) with each 110 features for all the epochs
def extract_x_y(epochs):
    epochs = epochs.pick(['F3', 'Fz', 'F4', 'FC1', 'FCz', 'FC2', 'C3', 'Cz', 'C4', 'CP1', 'CP2'])
    start_times = np.arange(-4.0, 3.001, 0.125)
    epochs.drop_bad(reject = dict(eeg=15e-5))
    for i, epoch in enumerate(epochs):
        raw = mne.io.RawArray(epoch, test.info)
        sliding_window = mne.make_fixed_length_epochs(raw, duration=1, overlap=0.875)
        X_features = sliding_window.reshape(sliding_window.shape[0], -1)
        y = np.array([0 if t < -1.5 else 1 for t in start_times])

        if i == 0:
            x_combined = X_features
            y_combined = y
        else:
            x_combined = np.concatenate([x_combined, X_features], axis=0)
            y_combined = np.concatenate([y_combined, y], axis=0)
    return x_combined, y_combined

#adjust for 1009 and 1029
def trial_correct_1009(y_true, y_pred, times):
    correct_in_window = any(
        (t >= -2.375 and t <= 1 and yt == 1 and yp == 1)
        for t, yt, yp in zip(times, y_true, y_pred)
    )
    incorrect_before = any(
        (t < -2.375 and yt == 0 and yp == 1)
        for t, yt, yp in zip(times, y_true, y_pred)
    )
    return correct_in_window and not incorrect_before

def trial_correct_1029(y_true, y_pred, times):
    correct_in_window = any(
        (t >= -1.375 and t <= 2 and yt == 1 and yp == 1)
        for t, yt, yp in zip(times, y_true, y_pred)
    )
    incorrect_before = any(
        (t < -1.375 and yt == 0 and yp == 1)
        for t, yt, yp in zip(times, y_true, y_pred)
    )
    return correct_in_window and not incorrect_before

## In Condition Training and Testing

Marker has to be selected manually, 1009 for the hand movement and 1029 for the start of VR movement marker


In [4]:
#Spliting train and test correctly
score_dict_lda = {'Joystick': [], 'Leaning':[], 'Omnideck':[], 'Walking':[]}
score_dict_svm = {'Joystick': [], 'Leaning':[], 'Omnideck':[], 'Walking':[]}
accuracy_dict_lda = {'Joystick': [], 'Leaning':[], 'Omnideck':[], 'Walking':[]}
accuracy_dict_svm = {'Joystick': [], 'Leaning':[], 'Omnideck':[], 'Walking':[]}
conditions = ['Joystick', 'Leaning', 'Omnideck', 'Walking']
marker = 1029
for cond in conditions:
    for p in range(1, 15):
        if p in [4] or (p == 7 and cond == 'Leaning') or (p == 7 and cond == 'Joystick') or (p == 3 and cond == 'Omnideck') or (p == 13 and cond == 'Walking'):
            score_dict_lda[cond].append(np.nan)
            score_dict_svm[cond].append(np.nan)
            accuracy_dict_lda[cond].append(np.nan)
            accuracy_dict_svm[cond].append(np.nan)
            continue

        path = f'epochs\\P{p:03d}\\'
        i = 1
        x_combined = []
        y_combined = []
        trial_data = []

        while i < 3:
            # P13 only has one walking condition
            if marker == 1009:
                filename = f'sub-P{p:03d}_ses-{cond}{i}_epochs_first_{marker}_epo.fif'     # Change to 1029
            else:
                filename = f'sub-P{p:03d}_ses-{cond}{i}_epochs_{marker}_epo.fif'
            epochs = mne.read_epochs(path + filename)
            epochs = epochs.pick(['F3', 'Fz', 'F4', 'FC1', 'FCz', 'FC2', 'C3', 'Cz', 'C4', 'CP1', 'CP2'])

            # For 1029: -5, 0.0001; For 1009: -6, -0.999
            if marker == 1009:
                start_times = np.arange(-6, 0.001, 0.125)
                epochs.crop(tmin=-6, tmax=1)
            else:
                start_times = np.arange(-5, 1.001, 0.125)
                epochs.crop(tmin=-5, tmax=2)
            epochs.resample(sfreq=10)

            ### Loop over all Epochs and create Sliding windows ###
            for j in range(len(epochs)):
                epoch = mrcp(epochs[j])
                # make_fixed_length_epochs can only be applied to raw data so I transform single epochs into raw by using the data and info from it
                raw = mne.io.RawArray(epoch.get_data(), epochs.info)
                sliding_window = mne.make_fixed_length_epochs(raw, duration=1, overlap=0.875)

                # Normalizing the features to euclidean length
                X = sliding_window.get_data()
                X_features = X.reshape(X.shape[0], -1)
                norms = np.linalg.norm(X_features, axis=1, keepdims=True)
                norms[norms == 0] = 1.0
                X_features = X_features / norms

                # Constructing the correct labels for y; a sliding window is regarded as pre-movement if more than half of the window is in pre-movement state
                y = []
                for t in start_times:
                    # For 1029: -1.375; For 1009: -2.375
                    if marker == 1009:
                        if t < -2.375:
                            y.append(0)
                        elif t > 0:
                            y.append(0)
                        else:
                            y.append(1)
                    else:
                        if t < -1.375:
                            y.append(0)
                        elif t > 1:
                            y.append(0)
                        else:
                            y.append(1)
                y = np.array(y)

                # Aggregating the epoch data
                x_combined.append(X_features)
                y_combined.append(y)
                trial_data.append((X_features, y, start_times))

            i += 1

        # combining the two sessions of a condition
        x_combined = np.concatenate(x_combined, axis=0)
        y_combined = np.concatenate(y_combined, axis=0)

        #initializing the classifiers, CV and scorer
        lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
        svm = SVC(kernel='rbf', probability=True)

        cv = StratifiedKFold(n_splits=5, shuffle=True)
        f05_scorer = make_scorer(fbeta_score, beta=0.5)

        # Perform window based classification
        scores_lda = cross_val_score(lda, x_combined, y_combined, cv=cv, scoring=f05_scorer)
        scores_svm = cross_val_score(svm, x_combined, y_combined, cv=cv, scoring=f05_scorer)

        score_dict_lda[cond].append(scores_lda.mean())
        score_dict_svm[cond].append(scores_svm.mean())

        ### Start of Accuracy Evaluation ###
        kf = KFold(n_splits=5, shuffle=True)

        accuracies_lda = []
        accuracies_svm = []
        for train_idx, test_idx in kf.split(trial_data):
            # Prepare data
            train_trials = [trial_data[i] for i in train_idx]
            test_trials = [trial_data[i] for i in test_idx]

            X_train = np.concatenate([X for X, y, t in train_trials], axis=0)
            y_train = np.concatenate([y for X, y, t in train_trials], axis=0)

            # Fit classifiers
            lda.fit(X_train, y_train)
            svm.fit(X_train, y_train)

            # Evaluate on test trials
            correct_lda = 0
            correct_svm = 0

            for X_trial, y_trial, t_trial in test_trials:
                y_pred_lda = lda.predict(X_trial)
                y_pred_svm = svm.predict(X_trial)
                if marker == 1009:
                    if trial_correct_1009(y_trial, y_pred_lda, t_trial):
                        correct_lda += 1
                    if trial_correct_1009(y_trial, y_pred_svm, t_trial):
                        correct_svm += 1
                else:
                    if trial_correct_1029(y_trial, y_pred_lda, t_trial):
                        correct_lda += 1
                    if trial_correct_1029(y_trial, y_pred_svm, t_trial):
                        correct_svm += 1

            fold_accuracy_lda = correct_lda / len(test_trials)
            fold_accuracy_svm = correct_svm / len(test_trials)

            accuracies_lda.append(fold_accuracy_lda)
            accuracies_svm.append(fold_accuracy_svm)

        # Store average accuracy over 5 folds
        accuracy_dict_lda[cond].append(np.mean(accuracies_lda))
        accuracy_dict_svm[cond].append(np.mean(accuracies_svm))
        print(f"Participant {p} | Condition {cond} | Mean LDA F0.5 Score: {scores_lda.mean():.3f} | Accuracy: {np.mean(accuracies_lda):.3f}")
        print(f"Participant {p} | Condition {cond} | Mean SVM F0.5 Score: {scores_svm.mean():.3f} | Accuracy: {np.mean(accuracies_svm):.3f}")
        print(f"Participant {p} | LDA CV {accuracies_lda} | SVM CV {accuracies_svm}")

# Print group means
for cond in conditions:
    print(f"{cond} | LDA Mean F0.5 Score: {np.nanmean(score_dict_lda[cond]):.3f} | LDA Accuracy: {np.nanmean(accuracy_dict_lda[cond]):.3f}")
    print(f"{cond} | SVM Mean F0.5 Score: {np.nanmean(score_dict_svm[cond]):.3f} | SVM Accuracy: {np.nanmean(accuracy_dict_svm[cond]):.3f}")

# Convert results to DataFrame for saving
participants = list(range(0, 14))
# Exclude participants that were skipped per condition
def get_valid_scores(score_dict, cond):
    valid_scores = []
    for idx, p in enumerate(participants):
        if (p in [4]) or (p == 3 and cond == 'Omnideck') or (p == 7 and cond == 'Leaning') or (p == 7 and cond == 'Joystick'):
            valid_scores.append(np.nan)
        else:
            valid_scores.append(score_dict[cond][len(valid_scores)])
    return valid_scores
# Create list of records (rows)
rows = []
for cond in conditions:
    for i, p in enumerate(participants):
        row = {
            "Participant": p+1,
            "Condition": cond,
            "LDA_F0.5_Score": score_dict_lda[cond][i],
            "SVM_F0.5_Score": score_dict_svm[cond][i],
            "LDA_Accuracy": accuracy_dict_lda[cond][i],
            "SVM_Accuracy": accuracy_dict_svm[cond][i]
        }
        rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows)

# Save to CSV
if marker == 1009:
    df.to_csv(f"results\\classification_results_single_cond_first_{marker}.csv", index=False)
else:
    df.to_csv(f"results\\classification_results_single_cond_{marker}.csv", index=False)
print("Results saved to classification_results.csv")


Participant 1 | Condition Joystick | Mean LDA F0.5 Score: 0.767 | Accuracy: 0.189
Participant 1 | Condition Joystick | Mean SVM F0.5 Score: 0.887 | Accuracy: 0.239
Participant 1 | LDA CV [0.125, 0.25, 0.2857142857142857, 0.0, 0.2857142857142857] | SVM CV [0.125, 0.5, 0.14285714285714285, 0.2857142857142857, 0.14285714285714285]
Participant 2 | Condition Joystick | Mean LDA F0.5 Score: 0.668 | Accuracy: 0.186
Participant 2 | Condition Joystick | Mean SVM F0.5 Score: 0.857 | Accuracy: 0.000
Participant 2 | LDA CV [0.25, 0.25, 0.0, 0.2857142857142857, 0.14285714285714285] | SVM CV [0.0, 0.0, 0.0, 0.0, 0.0]
Participant 3 | Condition Joystick | Mean LDA F0.5 Score: 0.674 | Accuracy: 0.161
Participant 3 | Condition Joystick | Mean SVM F0.5 Score: 0.873 | Accuracy: 0.154
Participant 3 | LDA CV [0.25, 0.0, 0.125, 0.2857142857142857, 0.14285714285714285] | SVM CV [0.375, 0.25, 0.0, 0.0, 0.14285714285714285]
Participant 5 | Condition Joystick | Mean LDA F0.5 Score: 0.686 | Accuracy: 0.214
Partic

In [16]:
# TODO: Chance level Estimate

Estimated Global Chance-Level Accuracy: 0.0000300000


3e-05

## Leave one Condition out Transferlearning

In [6]:
score_dict_lda = {'Joystick': [], 'Leaning':[], 'Omnideck':[], 'Walking':[]}
score_dict_svm = {'Joystick': [], 'Leaning':[], 'Omnideck':[], 'Walking':[]}
accuracy_dict_lda = {'Joystick': [], 'Leaning':[], 'Omnideck':[], 'Walking':[]}
accuracy_dict_svm = {'Joystick': [], 'Leaning':[], 'Omnideck':[], 'Walking':[]}

conditions = ['Joystick', 'Leaning', 'Omnideck', 'Walking']
participants = list(range(1, 15))
excluded = {
    (3, 'Omnideck'), (4, 'Joystick'), (7, 'Leaning'), (7, 'Joystick'), (13, 'Walking')
}
marker = 1029
for p in participants:
    for test_cond in conditions:
        ### Dealing with Missing data ###
        if (p, test_cond) in excluded:
            score_dict_lda[test_cond].append(np.nan)
            score_dict_svm[test_cond].append(np.nan)
            accuracy_dict_lda[test_cond].append(np.nan)
            accuracy_dict_svm[test_cond].append(np.nan)
            continue

        path = f'epochs\\P{p:03d}\\'
        x_train, y_train = [], []
        x_test, y_test = [], []
        test_trials = []

        for cond in conditions:
            if (p, cond) in excluded:
                continue
            for i in range(1, 3):
                if p == 13 and cond == 'Walking' and i == 2:
                    continue

                if marker == 1009:
                    filename = f'sub-P{p:03d}_ses-{cond}{i}_epochs_first_{marker}_epo.fif'
                else:
                    filename = f'sub-P{p:03d}_ses-{cond}{i}_epochs_{marker}_epo.fif'

                # If files does not exist
                try:
                    epochs = mne.read_epochs(path + filename, verbose='error')
                except FileNotFoundError:
                    print(f"File {filename} not found")
                    continue

                # Arranging the start times correctly based on marker; 1029 is delayed by one second
                epochs = epochs.pick(['F3', 'Fz', 'F4', 'FC1', 'FCz', 'FC2', 'C3', 'Cz', 'C4', 'CP1', 'CP2'])
                if marker == 1009:
                    start_times = np.arange(-6, 0.001, 0.125)
                    epochs.crop(tmin=-6, tmax=1)
                else:
                    start_times = np.arange(-5, 1.001, 0.125)
                    epochs.crop(tmin=-5, tmax=2)
                epochs.resample(sfreq=10)

                # Loop through all epochs create the sliding windows and labels
                for j in range(len(epochs)):
                    epoch = mrcp(epochs[j])
                    raw = mne.io.RawArray(epoch.get_data(), epochs.info)
                    sliding_window = mne.make_fixed_length_epochs(raw, duration=1, overlap=0.875)
                    X = sliding_window.get_data()
                    X_features = X.reshape(X.shape[0], -1)
                    norms = np.linalg.norm(X_features, axis=1, keepdims=True)
                    norms[norms == 0] = 1.0
                    X_features = X_features / norms

                    # cleaner code using list comprehensions
                    if marker == 1009:
                        y = [1 if -2.375 <= t <= 0 else 0 for t in start_times]
                    else:
                        y = [1 if -1.375 <= t <= 1 else 0 for t in start_times]
                    y = np.array(y)

                    # Aggregate the data based on the current test condition
                    if cond == test_cond:
                        x_test.append(X_features)
                        y_test.append(y)
                        test_trials.append((X_features, y, start_times))
                    else:
                        x_train.append(X_features)
                        y_train.append(y)

        # Deal with missing possible missing conditions if not flagged earlier
        if not x_train or not x_test:
            score_dict_lda[test_cond].append(np.nan)
            score_dict_svm[test_cond].append(np.nan)
            accuracy_dict_lda[test_cond].append(np.nan)
            accuracy_dict_svm[test_cond].append(np.nan)
            continue

        # Create the test and train data
        x_train = np.concatenate(x_train, axis=0)
        y_train = np.concatenate(y_train, axis=0)
        x_test_all = np.concatenate(x_test, axis=0)
        y_test_all = np.concatenate(y_test, axis=0)

        # Initialize Classifiers
        lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
        svm = SVC(kernel='rbf', probability=True)

        # Fit Classifiers
        lda.fit(x_train, y_train)
        svm.fit(x_train, y_train)

        # Predict test set and score based on F0.5-score
        y_pred_lda = lda.predict(x_test_all)
        y_pred_svm = svm.predict(x_test_all)

        f05_lda = fbeta_score(y_test_all, y_pred_lda, beta=0.5)
        f05_svm = fbeta_score(y_test_all, y_pred_svm, beta=0.5)

        score_dict_lda[test_cond].append(f05_lda)
        score_dict_svm[test_cond].append(f05_svm)

        # Trial-based accuracy; figured out += True works
        correct_lda = 0
        correct_svm = 0
        for X_trial, y_trial, t_trial in test_trials:
            pred_lda = lda.predict(X_trial)
            pred_svm = svm.predict(X_trial)
            if marker == 1009:
                correct_lda += trial_correct_1009(y_trial, pred_lda, t_trial)
                correct_svm += trial_correct_1009(y_trial, pred_svm, t_trial)
            else:
                correct_lda += trial_correct_1029(y_trial, pred_lda, t_trial)
                correct_svm += trial_correct_1029(y_trial, pred_svm, t_trial)

        total_trials = len(test_trials)
        accuracy_dict_lda[test_cond].append(correct_lda / total_trials)
        accuracy_dict_svm[test_cond].append(correct_svm / total_trials)

        print(f"Participant {p} | Test: {test_cond} | LDA F0.5: {f05_lda:.3f}, Acc: {correct_lda / total_trials:.3f}")
        print(f"Participant {p} | Test: {test_cond} | SVM F0.5: {f05_svm:.3f}, Acc: {correct_svm / total_trials:.3f}")

# Group mean results
for cond in conditions:
    print(f"{cond} | LDA Mean F0.5: {np.nanmean(score_dict_lda[cond]):.3f}, LDA Acc: {np.nanmean(accuracy_dict_lda[cond]):.3f}")
    print(f"{cond} | SVM Mean F0.5: {np.nanmean(score_dict_svm[cond]):.3f}, SVM Acc: {np.nanmean(accuracy_dict_svm[cond]):.3f}")

rows = []
for i, p in enumerate(participants):
    for cond in conditions:
        if i < len(score_dict_lda[cond]):
            rows.append({
                'Participant': p,
                'Test_Condition': cond,
                'LDA_F0.5': score_dict_lda[cond][i],
                'LDA_Accuracy': accuracy_dict_lda[cond][i],
                'SVM_F0.5': score_dict_svm[cond][i],
                'SVM_Accuracy': accuracy_dict_svm[cond][i]
            })

### Convert to DataFrame and save ###
df = pd.DataFrame(rows)
if marker == 1009:
    df.to_csv(f"results\\cross_condition_results_first_{marker}.csv", index=False)
else:
    df.to_csv(f"results\\cross_condition_results_{marker}.csv", index=False)
print("Results saved to 'cross_condition_results_first_1009.csv'")

Participant 1 | Test: Joystick | LDA F0.5: 0.518, Acc: 0.324
Participant 1 | Test: Joystick | SVM F0.5: 0.592, Acc: 0.189
Participant 1 | Test: Leaning | LDA F0.5: 0.386, Acc: 0.053
Participant 1 | Test: Leaning | SVM F0.5: 0.389, Acc: 0.026
Participant 1 | Test: Omnideck | LDA F0.5: 0.384, Acc: 0.081
Participant 1 | Test: Omnideck | SVM F0.5: 0.431, Acc: 0.135
Participant 1 | Test: Walking | LDA F0.5: 0.327, Acc: 0.081
Participant 1 | Test: Walking | SVM F0.5: 0.388, Acc: 0.135
Participant 2 | Test: Joystick | LDA F0.5: 0.325, Acc: 0.237
Participant 2 | Test: Joystick | SVM F0.5: 0.322, Acc: 0.105
Participant 2 | Test: Leaning | LDA F0.5: 0.386, Acc: 0.132
Participant 2 | Test: Leaning | SVM F0.5: 0.342, Acc: 0.053
Participant 2 | Test: Omnideck | LDA F0.5: 0.429, Acc: 0.184
Participant 2 | Test: Omnideck | SVM F0.5: 0.363, Acc: 0.026
Participant 2 | Test: Walking | LDA F0.5: 0.426, Acc: 0.105
Participant 2 | Test: Walking | SVM F0.5: 0.417, Acc: 0.105
Participant 3 | Test: Joystick |

## Walking --> Omnideck Transfer (and vice versa)

In [7]:
# Transfer learning from training on Omnideck and testing on real walking and the other way round
marker = 1009
results = []
participants = list(range(1, 15))
excluded = {
    (3, 'Omnideck'), (7, 'Leaning'), (7, 'Joystick'), (13, 'Walking')
}

for p in participants:
    for train_cond, test_cond in [('Walking', 'Omnideck'), ('Omnideck', 'Walking')]:
        if (p, train_cond) in excluded or (p, test_cond) in excluded or p == 4:
            results.append((p, train_cond, test_cond, np.nan, np.nan, np.nan, np.nan))
            continue

        path = f'epochs\\P{p:03d}\\'
        x_train, y_train = [], []
        x_test, y_test = [], []
        test_trials = []

        for cond, store in [(train_cond, 'train'), (test_cond, 'test')]:
            for i in range(1, 3):
                if p == 13 and cond == 'Walking' and i == 2:
                    continue
                if marker == 1009:
                    filename = f'sub-P{p:03d}_ses-{cond}{i}_epochs_first_{marker}_autoreject_epo.fif'
                else:
                    filename = f'sub-P{p:03d}_ses-{cond}{i}_epochs_{marker}_autoreject_epo.fif'
                try:
                    epochs = mne.read_epochs(path + filename, verbose='error')
                except FileNotFoundError:
                    continue
                epochs = epochs.pick(['F3', 'Fz', 'F4', 'FC1', 'FCz', 'FC2', 'C3', 'Cz', 'C4', 'CP1', 'CP2'])

                if marker == 1009:
                    start_times = np.arange(-6, 0.001, 0.125)
                    epochs.crop(tmin=-6, tmax=1)
                else:
                    start_times = np.arange(-5, 1.001, 0.125)
                    epochs.crop(tmin=-5, tmax=2)
                epochs.resample(sfreq=10)

                for j in range(len(epochs)):
                    epoch = mrcp(epochs[j])
                    raw = mne.io.RawArray(epoch.get_data(), epochs.info)
                    sliding_window = mne.make_fixed_length_epochs(raw, duration=1, overlap=0.875)
                    X = sliding_window.get_data()
                    X_features = X.reshape(X.shape[0], -1)
                    norms = np.linalg.norm(X_features, axis=1, keepdims=True)
                    norms[norms == 0] = 1.0
                    X_features = X_features / norms
                    if marker == 1009:
                        y = [1 if -2.375 <= t <= 0 else 0 for t in start_times]
                    else:
                        y = [1 if -1.375 <= t <= 1 else 0 for t in start_times]
                    y = np.array(y)

                    if store == 'train':
                        x_train.append(X_features)
                        y_train.append(y)
                    else:
                        x_test.append(X_features)
                        y_test.append(y)
                        test_trials.append((X_features, y, start_times))

        #if not x_train or not x_test:
        #    results.append((p, train_cond, test_cond, np.nan, np.nan, np.nan, np.nan))
        #    continue
        x_train = np.concatenate(x_train, axis=0)
        y_train = np.concatenate(y_train, axis=0)
        x_test_all = np.concatenate(x_test, axis=0)
        y_test_all = np.concatenate(y_test, axis=0)

        lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
        svm = SVC(kernel='rbf', probability=True)
        f05 = make_scorer(fbeta_score, beta=0.5)

        lda.fit(x_train, y_train)
        svm.fit(x_train, y_train)

        y_pred_lda = lda.predict(x_test_all)
        y_pred_svm = svm.predict(x_test_all)

        f05_lda = fbeta_score(y_test_all, y_pred_lda, beta=0.5)
        f05_svm = fbeta_score(y_test_all, y_pred_svm, beta=0.5)

        # Trial-based accuracy
        correct_lda = 0
        correct_svm = 0
        for X_trial, y_trial, t_trial in test_trials:
            pred_lda = lda.predict(X_trial)
            pred_svm = svm.predict(X_trial)
            if marker == 1009:
                correct_lda += trial_correct_1009(y_trial, pred_lda, t_trial)
                correct_svm += trial_correct_1009(y_trial, pred_svm, t_trial)
            else:
                correct_lda += trial_correct_1029(y_trial, pred_lda, t_trial)
                correct_svm += trial_correct_1029(y_trial, pred_svm, t_trial)

        total_trials = len(test_trials)
        acc_lda = correct_lda / total_trials
        acc_svm = correct_svm / total_trials

        results.append((p, train_cond, test_cond, f05_lda, acc_lda, f05_svm, acc_svm))

        print(f"P{p:02d} | Train: {train_cond} → Test: {test_cond} | "
              f"LDA F0.5: {f05_lda:.3f}, Acc: {acc_lda:.3f} | "
              f"SVM F0.5: {f05_svm:.3f}, Acc: {acc_svm:.3f}")

# Optional: print group means
for direction in [('Walking', 'Omnideck'), ('Omnideck', 'Walking')]:
    f05_lda_all = [r[3] for r in results if (r[1], r[2]) == direction and not np.isnan(r[3])]
    acc_lda_all = [r[4] for r in results if (r[1], r[2]) == direction and not np.isnan(r[4])]
    f05_svm_all = [r[5] for r in results if (r[1], r[2]) == direction and not np.isnan(r[5])]
    acc_svm_all = [r[6] for r in results if (r[1], r[2]) == direction and not np.isnan(r[6])]

    print(f"\nTrain: {direction[0]} → Test: {direction[1]}")
    print(f"LDA  Mean F0.5: {np.mean(f05_lda_all):.3f} | Mean Accuracy: {np.mean(acc_lda_all):.3f}")
    print(f"SVM  Mean F0.5: {np.mean(f05_svm_all):.3f} | Mean Accuracy: {np.mean(acc_svm_all):.3f}")

# Convert results to DataFrame
df_results = pd.DataFrame(results, columns=[
    'Participant', 'Train_Condition', 'Test_Condition',
    'LDA_F0.5', 'LDA_Accuracy', 'SVM_F0.5', 'SVM_Accuracy'
])

# Save to CSV
if marker == 1009:
    df_results.to_csv(f'results\\transfer_learning_omideck_walking_first_{marker}_results.csv', index=False)
else:
    df_results.to_csv(f'results\\transfer_learning_omideck_walking_{marker}_results.csv', index=False)
print("Results saved to 'transfer_learning_results.csv'")


P01 | Train: Walking → Test: Omnideck | LDA F0.5: 0.522, Acc: 0.222 | SVM F0.5: 0.408, Acc: 0.222
P01 | Train: Omnideck → Test: Walking | LDA F0.5: 0.421, Acc: 0.121 | SVM F0.5: 0.411, Acc: 0.091
P02 | Train: Walking → Test: Omnideck | LDA F0.5: 0.498, Acc: 0.273 | SVM F0.5: 0.530, Acc: 0.182
P02 | Train: Omnideck → Test: Walking | LDA F0.5: 0.607, Acc: 0.206 | SVM F0.5: 0.602, Acc: 0.059
P05 | Train: Walking → Test: Omnideck | LDA F0.5: 0.376, Acc: 0.057 | SVM F0.5: 0.379, Acc: 0.114
P05 | Train: Omnideck → Test: Walking | LDA F0.5: 0.355, Acc: 0.100 | SVM F0.5: 0.422, Acc: 0.033
P06 | Train: Walking → Test: Omnideck | LDA F0.5: 0.289, Acc: 0.121 | SVM F0.5: 0.226, Acc: 0.121
P06 | Train: Omnideck → Test: Walking | LDA F0.5: 0.184, Acc: 0.031 | SVM F0.5: 0.223, Acc: 0.031
P07 | Train: Walking → Test: Omnideck | LDA F0.5: 0.446, Acc: 0.028 | SVM F0.5: 0.481, Acc: 0.028
P07 | Train: Omnideck → Test: Walking | LDA F0.5: 0.396, Acc: 0.054 | SVM F0.5: 0.397, Acc: 0.054
P08 | Train: Walking

# Leftover Code from Cleanup

In [ ]:
# TODO: Make i a function with a parameter that can be 'trial', 'condition', 'subject' or 'global' that runs the classification or just assembles the data in given the scope
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, fbeta_score
from sklearn.svm import SVC
score_dict_lda = {'Joystick': [], 'Leaning':[], 'Omnideck':[], 'Walking':[]}
score_dict_svm = {'Joystick': [], 'Leaning':[], 'Omnideck':[], 'Walking':[]}
conditions = ['Joystick', 'Leaning', 'Omnideck', 'Walking']
for cond in conditions:
    for p in range(1,15):
        if p == 4 or p == 7 or p == 13:
            score_dict_lda[cond].append(np.nan)
            score_dict_svm[cond].append(np.nan)
            continue
        if p == 3 and cond == 'Omnideck':
            score_dict_lda[cond].append(np.nan)
            score_dict_svm[cond].append(np.nan)
            continue
        path = f'epochs\\P{p:03d}\\'
        i = 1
        while i < 3:
            filename = f'sub-P{p:03d}_ses-{cond}{i}_epochs_1029_epo.fif'
            epochs = mne.read_epochs(path+filename)
            epochs = epochs.pick(['F3', 'Fz', 'F4', 'FC1', 'FCz', 'FC2', 'C3', 'Cz', 'C4', 'CP1', 'CP2'])
            ar = AutoReject(n_jobs=8, verbose=False)
            epochs = ar.fit_transform(epochs)
            start_times = np.arange(-6, 0.001, 0.125)
            nr_epochs = len(epochs)
            epochs.crop(tmin=-6, tmax=1)
            epochs.resample(sfreq=10)

            for j in range(len(epochs)):
                epoch = mrcp(epochs[j])
                raw = mne.io.RawArray(epoch.get_data(), epochs.info)
                sliding_window = mne.make_fixed_length_epochs(raw, duration=1, overlap=0.875)
                X = sliding_window.get_data()
                X_features = X.reshape(X.shape[0], -1)
                norms = np.linalg.norm(X_features, axis=1, keepdims=True)
                norms[norms == 0] = 1.0  # prevent division by zero
                X_features = X_features / norms
                y = []
                for t in start_times:
                    if t < -0.5:
                        y.append(0)
                    elif t > 1:
                        y.append(0)
                    else:
                        y.append(1)
                y = np.array(y)
                if i == 1 and j == 0:
                    x_combined = X_features
                    y_combined = y
                else:
                    x_combined = np.concatenate([x_combined, X_features], axis=0)
                    y_combined = np.concatenate([y_combined, y], axis=0)
            i += 1
        lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
        svm = SVC(kernel='rbf', probability=True, random_state=42)

        # Stratified 5-fold cross-validation
        cv = StratifiedKFold(n_splits=10, shuffle=True)
        f05_scorer = make_scorer(fbeta_score, beta=0.5)
        # Cross-validation scoring (you can change 'accuracy' to 'roc_auc', etc.)
        scores_lda = cross_val_score(lda, x_combined, y_combined, cv=cv, scoring=f05_scorer)
        scores_svm = cross_val_score(svm, x_combined, y_combined, cv=cv, scoring=f05_scorer)
        score_dict_lda[cond].append(scores_lda.mean())
        score_dict_svm[cond].append(scores_svm.mean())
        print(f"Participant {p} | Condition {cond} | Mean LDA F0.5 Score:", scores_lda.mean())
        print(f"Participant {p} | Condition {cond} | Mean SVM F0.5 Score:", scores_svm.mean())
for cond in conditions:
    print(f"{cond} Mean F0.5 Score:", np.nanmean(np.array(score_dict_lda[cond])))
    print(f"{cond} Mean F0.5 Score:", np.nanmean(np.array(score_dict_svm[cond])))


In [ ]:

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, fbeta_score
from sklearn.svm import SVC
import numpy as np
import mne
from autoreject import AutoReject

score_dict_lda = {'Joystick': [], 'Leaning':[], 'Omnideck':[], 'Walking':[]}
score_dict_svm = {'Joystick': [], 'Leaning':[], 'Omnideck':[], 'Walking':[]}
accuracy_dict_lda = {'Joystick': [], 'Leaning':[], 'Omnideck':[], 'Walking':[]}
accuracy_dict_svm = {'Joystick': [], 'Leaning':[], 'Omnideck':[], 'Walking':[]}

conditions = ['Joystick', 'Leaning', 'Omnideck', 'Walking']
for cond in conditions:
    for p in range(1, 15):
        if p in [4] or (p == 7 and cond == 'Leaning') or (p == 7 and cond == 'Joystick'):
            score_dict_lda[cond].append(np.nan)
            score_dict_svm[cond].append(np.nan)
            accuracy_dict_lda[cond].append(np.nan)
            accuracy_dict_svm[cond].append(np.nan)
            continue

        path = f'epochs\\P{p:03d}\\'
        i = 1
        x_combined = []
        y_combined = []
        trial_data = []

        while i < 3:
            # P13 only has one walking condition
            if (p == 13 and cond == 'Walking' and i == 2) or (p == 3 and cond == 'Omnideck' and i == 2):
                i+=1
                continue
            filename = f'sub-P{p:03d}_ses-{cond}{i}_epochs_first_1009_epo.fif'      # Change to 1029
            epochs = mne.read_epochs(path + filename)
            epochs = epochs.pick(['F3', 'Fz', 'F4', 'FC1', 'FCz', 'FC2', 'C3', 'Cz', 'C4', 'CP1', 'CP2'])
            ar = AutoReject(n_jobs=8, verbose=False)
            epochs = ar.fit_transform(epochs)
            # For 1029: -5, 0.0001; For 1009: -6, -0.999
            start_times = np.arange(-6, -0.999, 0.125)
            epochs.crop(tmin=-6, tmax=0)
            epochs.resample(sfreq=10)

            for j in range(len(epochs)):
                epoch = mrcp(epochs[j])
                raw = mne.io.RawArray(epoch.get_data(), epochs.info)
                sliding_window = mne.make_fixed_length_epochs(raw, duration=1, overlap=0.875)
                X = sliding_window.get_data()
                X_features = X.reshape(X.shape[0], -1)
                norms = np.linalg.norm(X_features, axis=1, keepdims=True)
                norms[norms == 0] = 1.0
                X_features = X_features / norms
                y = []
                for t in start_times:
                    # For 1029: -0.875; For 1009: -1.87
                    if t < -1.875:
                        y.append(0)
                    elif t > 0:
                        y.append(0)
                    else:
                        y.append(1)
                y = np.array(y)

                x_combined.append(X_features)
                y_combined.append(y)
                trial_data.append((X_features, y, start_times))

            i += 1

        x_combined = np.concatenate(x_combined, axis=0)
        y_combined = np.concatenate(y_combined, axis=0)

        lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
        svm = SVC(kernel='rbf', probability=True)

        cv = StratifiedKFold(n_splits=5, shuffle=True)
        f05_scorer = make_scorer(fbeta_score, beta=0.5)

        scores_lda = cross_val_score(lda, x_combined, y_combined, cv=cv, scoring=f05_scorer)
        scores_svm = cross_val_score(svm, x_combined, y_combined, cv=cv, scoring=f05_scorer)

        score_dict_lda[cond].append(scores_lda.mean())
        score_dict_svm[cond].append(scores_svm.mean())

        # Fit classifiers for trial-wise accuracy
        lda.fit(x_combined, y_combined)
        svm.fit(x_combined, y_combined)

        correct_trials_lda = 0
        correct_trials_svm = 0

        for X_trial, y_trial, t_trial in trial_data:
            y_pred_lda = lda.predict(X_trial)
            y_pred_svm = svm.predict(X_trial)

            # Accuracy
            if trial_correct(y_trial, y_pred_lda, t_trial):
                correct_trials_lda += 1
            if trial_correct(y_trial, y_pred_svm, t_trial):
                correct_trials_svm += 1

        total_trials = len(trial_data)
        accuracy_dict_lda[cond].append(correct_trials_lda / total_trials)
        accuracy_dict_svm[cond].append(correct_trials_svm / total_trials)

        print(f"Participant {p} | Condition {cond} | Mean LDA F0.5 Score: {scores_lda.mean():.3f} | Accuracy: {correct_trials_lda / total_trials:.3f}")
        print(f"Participant {p} | Condition {cond} | Mean SVM F0.5 Score: {scores_svm.mean():.3f} | Accuracy: {correct_trials_svm / total_trials:.3f}")

# Print group means
for cond in conditions:
    print(f"{cond} | LDA Mean F0.5 Score: {np.nanmean(score_dict_lda[cond]):.3f} | LDA Accuracy: {np.nanmean(accuracy_dict_lda[cond]):.3f}")
    print(f"{cond} | SVM Mean F0.5 Score: {np.nanmean(score_dict_svm[cond]):.3f} | SVM Accuracy: {np.nanmean(accuracy_dict_svm[cond]):.3f}")
